---
## Refining Aggregations

Working from the csv obtained after crawling *brewtoad* recipes, we perform several sets of **aggregations** and corrections to obtain clean **datasets** to work with.



---

### Step 1: Importing libraries and loading data

In [2]:
import numpy as np
import pandas as pd

In [3]:
%run utils.py

In [4]:
# Loading data from csv
data = load_from_csv(1, 10)

In [5]:
# Formatting data
data = format_data(data)

In [6]:
recipes = data['recipes']
styles = data['styles']
fermentables = data['fermentables']
hops = data['hops']
yeasts = data['yeasts']
miscs = data['miscs']

### Step 3: Normalizing quantities (except for Miscs)

In [7]:
recipes[:3]

,beer_id,beer_name,mash_type,brewer,batch_size,boil_size,efficiency
0,oatmeal-chocolate-milk-stout-1,Oatmeal Chocolate Milk Stout,All Grain,mattc,18.9271,23.6588,75
1,citripa,Citripa,All Grain,Marcel Woodman,20.8198,23.6588,75
2,janets-brown-ale-extract,Janet's Brown Ale (Extract),Partial Mash,Craig,18.9271,23.6588,75


In [8]:
cols = fermentables.keys()
cols = list(cols)
fermentables = pd.merge(recipes, fermentables, on=['beer_id', 'beer_name'], how='left')
fermentables = fermentables[fermentables.batch_size != 0]
fermentables['weight_rel'] = fermentables['amount'] / fermentables['batch_size']
cols.append('weight_rel')
fermentables = fermentables[cols]

In [9]:
cols = hops.keys()
cols = list(cols)
hops = pd.merge(recipes, hops, on=['beer_id', 'beer_name'], how='left')
hops = hops[hops.batch_size != 0]
hops['weight_rel'] = hops['amount'] / hops['batch_size']
cols.append('weight_rel')
hops = hops[cols]

In [10]:
hops_time = hops.groupby('time')

In [11]:
count = hops_time.count().reset_index()

In [12]:
count.columns

Index([u'time', u'beer_id', u'beer_name', u'name', u'origin', u'alpha',
       u'beta', u'amount', u'use', u'form', u'notes', u'weight_rel'],
      dtype='object')

In [13]:
count['count'] = count['beer_id'] / len(recipes)
count = count[['time', 'count']]

In [14]:
count_boil = count[count['time'] <= 100]

In [15]:
count_boil = count_boil[count_boil[]]

SyntaxError: invalid syntax (<ipython-input-15-e17ada93e81c>, line 1)

In [16]:
#count_boil.sort_values('count', ascending=False)

In [17]:
#hops_time.count().sort_values('time')

### Step 2: Grouping data

In [18]:
# Grouping data on beer_id
fermentables_grouped = fermentables.groupby('beer_id')
hops_grouped = hops.groupby('beer_id')
yeasts_grouped = yeasts.groupby('beer_id')
miscs_groupes = miscs.groupby('beer_id')

In [19]:
l = fermentables_grouped.groups.values()

In [20]:
max_nb_hops = max([len(x) for x in l])

In [21]:
range(0, max_nb_hops)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [42]:
params = ['time', 'name', 'weight_rel', 'alpha']
groups = hops_grouped.groups
recipes_hops_dfs = []
for group in groups:
    l = []
    for elt in groups[group]:
        d = {x: '' for x in params}
        hop = hops.loc[elt]
        for param in params:
            d[param] = hop[param]
        if hop['use'] == 'Dry Hop':
            d['time'] = 
        l.append(d)
        recipes_hops_dfs.append(pd.DataFrame(l))


In [43]:
recipes_hops_dfs[3]

,alpha,name,time,weight_rel
0,10.0,Centennial (US),60.0,1.479365
1,10.0,Centennial (US),20.0,1.479365
2,10.0,Centennial (US),5.0,2.958730
3,10.0,Centennial (US),18720.0,1.479365


In [35]:
dict(hops.loc[groups[tmp][0]])

{'alpha': 8.9000000000000004,
 'amount': 84.0,
 'beer_id': 'barrel-aged-rios',
 'beer_name': 'Barrel Aged RIOS',
 'beta': '',
 'form': 'Pellet',
 'name': 'Challenger (UK)',
 'notes': 'English hop. Introduced in 1972. Very popular dual-purpose hop in English ales. Used in many traditional English Bitters. When used for bittering, Challenger can impart a pleasant and complex marmalade/toffee/citrus flavour, which enhances strong ales. Substitutes: East Kent Goldings, Phoenix, Styrian Goldings, British Columbian Goldings.',
 'origin': 'United Kingdom',
 'time': 60.0,
 'use': 'Boil',
 'weight_rel': 4.034627713005974}

#### IBU - Hop Study 

In [327]:
hop_reduced_dict_keys = ['alpha', 'weight_rel', 'time', 'dry_hop_flag']
params = ['beer_id']
for p in range(0, max_nb_hops):
    for param in hop_reduced_dict_keys:
        params.append(param + ' (H{0})'.format(p))

In [328]:
def f(l):
    d = {}
    k = 0
    for el in l:
        d['alpha' + ' (H{0})'.format(k)] = el['alpha']
        d['weight_rel' + ' (H{0})'.format(k)] = el['weight_rel']
        d['time' + ' (H{0})'.format(k)] = el['time']
        d['dry_hop_flag' + ' (H{0})'.format(k)] = el['use'] == 'Dry Hop'
        k += 1
    return d

In [329]:
recipes_list = []
for key in hops_grouped.groups.keys():
    hops_list = hops_grouped.groups[key] 
    hops_list_f = []
    for hop_nb in hops_list:
        hop = hops.loc[hop_nb]
        hops_list_f.append(dict(hop))
    recipe_dict = f(hops_list_f)
    recipe_dict['beer_id'] = key
    recipes_list.append(recipe_dict)
hops_steps = pd.DataFrame(recipes_list, columns = params)

In [330]:
#hops_steps

In [331]:
# Getting IBU

In [343]:
tmp = pd.merge(hops_steps, styles, on='beer_id', how='inner')

In [344]:
len(tmp)

29937

In [346]:
# !!!!! Approximation grossière
tmp['IBU'] = tmp['IBU_min'] # + tmp['IBU_max']) / 2 

In [347]:
params.pop(0)

'beer_id'

In [384]:
df = tmp[params + ['IBU']]

In [385]:
df = df.fillna(0)

In [386]:
df['random'] = np.random.uniform(0, 1, len(df))

In [390]:
df

,alpha (H0),weight_rel (H0),time (H0),dry_hop_flag (H0),alpha (H1),weight_rel (H1),time (H1),dry_hop_flag (H1),alpha (H2),weight_rel (H2),...,alpha (H14),weight_rel (H14),time (H14),dry_hop_flag (H14),alpha (H15),weight_rel (H15),time (H15),dry_hop_flag (H15),IBU,random
0,10.00,1.479365,60.0,False,10.00,1.479365,20.0,False,10.00,2.958730,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0,40,0.976794
1,10.40,1.109524,60.0,False,10.40,1.479365,20.0,False,10.40,2.958730,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0,40,0.043554
2,15.00,1.681095,60.0,False,6.50,0.672438,30.0,False,6.50,2.689752,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0,40,0.644006
3,8.10,1.479365,60.0,False,8.10,1.479365,20.0,False,8.10,2.958730,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0,40,0.575089
4,13.00,0.718138,60.0,False,13.00,0.718138,30.0,False,13.00,0.718138,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0,30,0.173929
5,7.20,0.739682,60.0,False,7.20,0.739682,15.0,False,7.20,0.739682,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0,20,0.540895
6,7.00,2.689752,20.0,False,7.00,1.344876,10.0,False,7.00,2.689752,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0,30,0.882062
7,7.50,1.000000,60.0,False,5.25,0.600000,5.0,False,0.00,0.000000,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0,18,0.516093
8,8.50,1.344876,60.0,False,3.25,0.672438,30.0,False,3.25,0.672438,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0,20,0.685461
9,7.00,1.250000,60.0,False,7.00,0.750000,10.0,False,7.00,3.000000,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0,22,0.598149


In [389]:
train = df[df.random < 0.75]
train_y = np.array(train['IBU'])
train = np.array(train[params])
test = df[df.random >= 0.75]
test = np.array(test['IBU'])
test = np.array(test[params])

/Users/ulysse/Virtualenvs/crawl/lib/python2.7/site-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning: non integer (and non boolean) array-likes will not be accepted as indices in the future


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [363]:
from sklearn.linear_model import LinearRegression

In [364]:
regr = LinearRegression()

In [365]:
regr.fit(train, train_y)

ValueError: could not convert string to float: 